# Implementing C3D Model for Video Classification

In [1]:
%load_ext autoreload
%autoreload 2

from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import pickle
import glob
import cv2
import sys
import os
import time

from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
import seaborn as sns

 
# adding video-download folder to the system path
sys.path.insert(0, '/workspace/youtube-humpback-whale-classifier/video-download')
 
# importing read_frames_hdf5 function
from hdf5_data_loading import read_frames_hdf5

#ngc workspace path (where we keep our data)
workspace_path = '/mount/data'

In [2]:
import wandb

#start wandb session for metric logging
wandb.login() 

wandb.init(project="whale-classification-inception")

wandb.run.name = "c3d-training"

wandb: Currently logged in as: micheller (epg). Use `wandb login --relogin` to force relogin


In [3]:
print("Num GPUs available: ", len(tf.config.list_physical_devices('GPU'))) #1 if we select GPU mode in Colab Notebook, 0 if running on local machine

Num GPUs available:  4


In [3]:
#limit GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

4 Physical GPUs, 4 Logical GPUs


2022-07-26 18:56:41.114282: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-26 18:56:46.104443: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14649 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:85:00.0, compute capability: 7.0
2022-07-26 18:56:46.106576: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14649 MB memory:  -> device: 1, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:86:00.0, compute capability: 7.0
2022-07-26 18:56:46.110866: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/repli

## Load Dataset

In [4]:
#load dataset in
data = pd.read_csv(workspace_path + '/downloaded_videos.csv')
y = data.pop('relevant')
X = data

## Load Frames

In [5]:
#load in frames for all videos
start = time.time()

N = X.shape[0] #number of videos in our dataset
videos = np.empty((N, 30, 224, 224, 3), dtype=np.uint8)
labels = np.empty(N, dtype = np.uint8)

for i, video in enumerate(list(X.renamed_title)):
    if i % 50 == 0:
        print(f'Loading frames for video {i}...')
        
    clip_name = video.replace("_", "_clip_").replace(".mp4", "")
    frames, frame_labels = read_frames_hdf5(clip_name) #returns frames array of shape (461, 224, 224, 3)
    
    videos[i, ...] = frames[15:45] #shortened videos bc of memory issue - each video is reduced to 30 frames
    labels[i] = frame_labels[0] #all frames have the same label since label is assigned to overall video

stop = time.time()
print(f'Done loading frames in {stop-start} seconds.')
videos.shape

Loading frames for video 0...
Loading frames for video 50...
Loading frames for video 100...
Loading frames for video 150...
Loading frames for video 200...
Loading frames for video 250...
Loading frames for video 300...
Loading frames for video 350...
Done loading frames in 68.86091804504395 seconds.


(364, 30, 224, 224, 3)

## Split Data

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

train_index = list(X_train.index)
test_index = list(X_test.index)

# index data accordingly
train_videos, train_labels = videos[train_index], labels[train_index]
test_videos, test_labels = videos[test_index], labels[test_index]

# reshape label arrays as horizontal arrays
train_labels = np.reshape(train_labels, (train_labels.shape[0], 1))
test_labels = np.reshape(test_labels, (test_labels.shape[0], 1))

In [7]:
print(train_videos.shape)
print(test_videos.shape)

(291, 30, 224, 224, 3)
(73, 30, 224, 224, 3)


## Converting Data into TF Format

In [8]:
# train_tf = tf.data.Dataset.from_tensor_slices((train_videos, train_labels)).batch(10)
# train_tf

2022-07-25 23:37:27.760440: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-25 23:37:32.468881: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14649 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:85:00.0, compute capability: 7.0
2022-07-25 23:37:32.477385: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14649 MB memory:  -> device: 1, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:86:00.0, compute capability: 7.0
2022-07-25 23:37:32.483364: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/repli

<BatchDataset element_spec=(TensorSpec(shape=(None, 30, 224, 224, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 1), dtype=tf.uint8, name=None))>

`GPU keeps allocating 15516MiB everytime we load any sort of data, which leads to running out of memory at model training`

## Build C3D Video Classification Model

Resources
- https://towardsdatascience.com/step-by-step-implementation-3d-convolutional-neural-network-in-keras-12efbdd7b130
- https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7410867&tag=1

In [12]:
# import os
# os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
# print(os.getenv('TF_GPU_ALLOCATOR'))

In [8]:
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# fake_data = tf.random.normal((10, 30, 224, 224, 3))
# fake_label = tf.random.normal((10,))
# dataset = tf.data.Dataset.from_tensor_slices((fake_data, fake_label)).batch(1)
# print("Data ready")

inputs = keras.Input(shape=(30, 224, 224, 3)) #full shape w/ batch dim = (None, 30, 224, 224, 3)
x = Conv3D(64, kernel_size=(3,3,3), strides=(1,1,1))(inputs)
x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1,1,1))(x)
x = Conv3D(128, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)
x = Conv3D(256, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = Conv3D(256, kernel_size=(3,3,3), strides=(1,1,1))(x)
# x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)
# x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
# x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
# x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)
# x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
# x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)

x = Flatten()(x)
output = Dense(2, activation="softmax")(x)

c3d_model = keras.Model(inputs, output)
c3d_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# c3d_model.summary()

history = c3d_model.fit(train_videos, train_labels,
                        epochs = 5,
                        verbose= 1)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPUs will likely run quickly with dtype policy mixed_float16 as they all have compute capability of at least 7.0
Epoch 1/5


2022-07-26 18:59:17.663543: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2022-07-26 18:59:18.952483: I tensorflow/stream_executor/cuda/cuda_dnn.cc:379] Loaded cuDNN version 8400


In [18]:
wandb.finish()